<a href="https://colab.research.google.com/github/NikitaSavvin2000/Load-Balance/blob/NSavvin_develop/poisson_new.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [29]:
import pandas as pd
import numpy as np
import random
format_string = '%Y-%m-%d %H:%M:%S%z'
from datetime import datetime

def poisson_value(date_with_nan, df, time_window_minutes=5):
    date_with_nan = datetime.strptime(date_with_nan, format_string)
    # Создайте маску, чтобы выбрать все строки, которые соответствуют условиям месяца, дня, часа и минуты.
    mask = ((df['time'].dt.month == date_with_nan.month) &
            (df['time'].dt.day == date_with_nan.day) &
            (df['time'].dt.hour == date_with_nan.hour) &
            (df['time'].dt.minute >= date_with_nan.minute - time_window_minutes) &
            (df['time'].dt.minute <= date_with_nan.minute + time_window_minutes))
    # Отфильтруйте строки, чтобы исключить NaN значения в столбце 'P_l'.
    non_nan_values = df.loc[mask, 'P_l'].dropna()
    matching_indices = non_nan_values.index.tolist()
    # Проверьте, что есть хотя бы одно значение без NaN.
    if non_nan_values.empty:
        return []  # Вернуть пустой список, если нет подходящих значений.

    # Вычислите среднее значение для отфильтрованных значений.
    mean_value = non_nan_values.mean()

    # Генерируйте случайные значения Пуассона на основе среднего значения.
    count_index = 100
    s = np.random.poisson(mean_value, count_index).tolist()
    index = random.randint(0, 99)
    poisson_value = s[index]
    return poisson_value


def calculate_mape(df_orig, df_test, drop_index):
    df_test.reset_index(inplace=True)
    df_orig.reset_index(inplace=True)
    df_init = df_orig[df_orig['time'].isin(drop_index)]
    df_mape = df_test[df_test['time'].isin(drop_index)]
    mape = (abs((df_init['P_l'] - df_mape['P_l']) / df_mape['P_l']) * 100).mean()
    return mape


In [30]:
df=pd.read_csv(r'/content/filled_P_l.csv')

In [33]:

percentage_of_missing_values = 0.01  # например, 10%
total_values = len(df)
num_values_to_remove = int(total_values * percentage_of_missing_values / 100)
drop_index = random.sample(df.index.tolist(), num_values_to_remove)
df_test = df.copy()
df_test.loc[drop_index, 'P_l'] = np.nan
date_with_nan_list = df_test.loc[df_test['P_l'].isna(), 'time'].tolist()


In [34]:
print(len(date_with_nan_list))

141251


In [35]:
df_test['time'] = pd.to_datetime(df['time'], format=format_string)

In [ ]:
from tqdm import tqdm


for date_with_nan in tqdm(date_with_nan_list):
  random_poisson_value = poisson_value(date_with_nan, df_test, time_window_minutes=5)
  df_test.loc[df_test['time'] == date_with_nan, 'P_l'] = random_poisson_value


  2%|▏         | 3016/141251 [20:47<18:33:33,  2.07it/s]

In [ ]:
mape = calculate_mape(df, df_test, drop_index)
print(mape)